In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import math
import os

def load_stock_data_from_csv(csv_path):
    print(f"Loading data from: {csv_path}")

    df = pd.read_csv(csv_path)

    for _ in df.columns:
        df.columns = df.columns.str.strip()

    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')

    df = df.sort_index()

    if df.isnull().sum().sum() > 0:
        df = df.fillna(method='ffill')
        df = df.fillna(method='bfill')

    return df

def prepare_multivariate_lstm_data(data, sequence_length=60):
    features = ['Open', 'High', 'Low', 'Close', 'Volume']

    scalers = {}
    scaled_data = pd.DataFrame(index=data.index)

    for feature in features:
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data[feature] = scaler.fit_transform(data[feature].values.reshape(-1, 1)).flatten()
        scalers[feature] = scaler

    train_size = int(len(scaled_data) * 0.8)
    train_data = scaled_data.iloc[:train_size]
    test_data = scaled_data.iloc[train_size-sequence_length:]

    X_train, y_train = [], []
    for i in range(sequence_length, len(train_data)):
        X_train.append(train_data.iloc[i-sequence_length:i].values)
        y_train.append(scaled_data.iloc[i]['Close'])

    X_train, y_train = np.array(X_train), np.array(y_train)

    X_test, y_test = [], []
    for i in range(sequence_length, len(test_data)):
        X_test.append(test_data.iloc[i-sequence_length:i].values)
        y_test.append(scaled_data.iloc[train_size-sequence_length+i]['Close'])

    X_test, y_test = np.array(X_test), np.array(y_test)

    return X_train, y_train, X_test, y_test, scalers

def build_multivariate_lstm_model(X_train, y_train, X_test, y_test):
    model = Sequential()

    model.add(LSTM(units=100, return_sequences=True,
                  input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.2))

    model.add(LSTM(units=100, return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(units=100, return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(units=50))
    model.add(Dropout(0.2))

    model.add(Dense(units=1))

    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(
        X_train, y_train,
        epochs=200,
        batch_size=32,
        validation_data=(X_test, y_test),
        callbacks=[early_stop],
        verbose=1
    )

    return model, history

def evaluate_model(model, X_test, y_test, close_scaler):
    predictions = model.predict(X_test)

    y_test_inv = close_scaler.inverse_transform(y_test.reshape(-1, 1))
    predictions_inv = close_scaler.inverse_transform(predictions)

    rmse = math.sqrt(mean_squared_error(y_test_inv, predictions_inv))
    mae = mean_absolute_error(y_test_inv, predictions_inv)
    mape = np.mean(np.abs((y_test_inv - predictions_inv) / y_test_inv)) * 100
    r2 = r2_score(y_test_inv, predictions_inv) * 100

    print(f"RMSE: ${rmse:.2f}")
    print(f"MAE: ${mae:.2f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"R-Squared: {r2}")

    return predictions_inv, y_test_inv

def plot_predictions(predictions, actual, original_dates, title="Nuclear Stock Price Prediction"):
    plt.figure(figsize=(14, 7))
    plt.plot(original_dates, actual, color='blue', label='Actual Close Price')
    plt.plot(original_dates, predictions, color='red', label='Predicted Close Price')
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('Stock Price ($)')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def predict_future(model, last_sequence, scalers, features, days_to_predict=30):
    future_predictions = []
    current_sequence = last_sequence.reshape(1, last_sequence.shape[0], last_sequence.shape[1])

    for _ in range(days_to_predict):
        next_day_close_pred = model.predict(current_sequence)[0, 0]
        future_predictions.append(next_day_close_pred)

        new_row = np.zeros(len(features))

        close_idx = features.index('Close')
        open_idx = features.index('Open')
        high_idx = features.index('High')
        low_idx = features.index('Low')
        volume_idx = features.index('Volume')

        new_row[close_idx] = next_day_close_pred

        new_row[open_idx] = current_sequence[0, -1, close_idx]

        new_row[high_idx] = next_day_close_pred * 1.01
        new_row[low_idx] = next_day_close_pred * 0.99

        new_row[volume_idx] = current_sequence[0, -1, volume_idx]

        current_sequence = np.append(current_sequence[:, 1:, :],
                                    [new_row.reshape(1, -1)],
                                    axis=1)

    future_predictions = np.array(future_predictions).reshape(-1, 1)
    future_predictions = scalers['Close'].inverse_transform(future_predictions)

    return future_predictions

def run_nuclear_stock_prediction(csv_path, sequence_length=60):
    df = load_stock_data_from_csv(csv_path)

    X_train, y_train, X_test, y_test, scalers = prepare_multivariate_lstm_data(
        df, sequence_length=sequence_length
    )

    model, history = build_multivariate_lstm_model(X_train, y_train, X_test, y_test)
    predictions, actual = evaluate_model(model, X_test, y_test, scalers['Close'])

    train_size = int(len(df) * 0.8)
    test_dates = df.index[train_size:train_size+len(predictions)]

    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

    plot_predictions(predictions, actual, test_dates, "Nuclear Stock Price Prediction")

    last_sequence = X_test[-1]
    future_preds = predict_future(model, last_sequence, scalers,
                                 df.columns.tolist(), days_to_predict=30)

    last_date = df.index[-1]
    future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=30, freq='B')

    plt.figure(figsize=(14, 7))
    plt.plot(future_dates, future_preds, color='red', label='Future Predictions')
    plt.title('Nuclear Stock Future Prediction (30 Trading Days)')
    plt.xlabel('Date')
    plt.ylabel('Stock Price ($)')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    return model, scalers, df

if __name__ == "__main__":
    csv_file = "./StockMarket/NexGen Energy Ltd..csv"
    model, scalers, df = run_nuclear_stock_prediction(csv_file)